# Guided Projects Artificial Intelligence & Machine Learning
## Guided Projects: Natural Language Processing 
### Applications in Natural Language Processing
Natural Language Processing (NLP) is concerned with machine-human collaboration. It 
helps computers read, interpret, and understand the human language so that machines 
can carry our repetitive and high-volume tasks. It is the field of Artificial Intelligence (AI) 
that focuses on quantifying human language to make it intelligible to machines by 
combining the power of linguistics and computer science to study the rules and structure
of language and create intelligent systems.
### Question:
Using NLP we can easily analyse any given text. The steps involved for such an analysis are 
tokenization, pre processing each word and then finally vectorising each of them. One of the 
most common and easy to implement vectorisation algorithm is BoW. Using BoW and NLTK 
for processing, implement a simple spam filter that marks all the spam texts as dangerous.

In [2]:
! pip install nltk

In [63]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

In [64]:
nltk.download('stopwords')
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
data = pd.read_csv('../22. Applications in Natural Language Processing/spam.csv')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [15]:
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"class", "v2":"text"})
data.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
data['length'] = data['text'].apply(len)
data.head()

,class,text,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [18]:
data['class'].value_counts()

ham     4825
spam     747
Name: class, dtype: int64

In [20]:
stop_words = nltk.corpus.stopwords.words('english')
stop_words[:5]

['i', 'me', 'my', 'myself', 'we']

In [33]:
def pre_process(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    words = ""
    for i in text:
            words += (ps.stem(i))+" "
    return words

In [38]:
text = data['text'][0]
text

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [40]:
text = text.translate(str.maketrans('', '', string.punctuation))
text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
text[:5]

['Go', 'jurong', 'point', 'crazy', 'Available']

In [39]:
pre_process(text)

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat '

In [65]:
data['text']

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: text, Length: 5572, dtype: object

In [66]:
textFeatures = data['text'].copy()
textFeatures = textFeatures.apply(pre_process)
textFeatures[:5]

0    go jurong point crazi avail bugi n great world...
1                               ok lar joke wif u oni 
2    free entri 2 wkli comp win fa cup final tkt 21...
3                 u dun say earli hor u c alreadi say 
4           nah dont think goe usf live around though 
Name: text, dtype: object

In [53]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
features = vectorizer.fit_transform(textFeatures)
features.shape

(5572, 39213)

In [51]:
features_train, features_test, labels_train, labels_test = train_test_split(features, data['class'], 
                                                                            test_size=0.3, random_state=111)

In [57]:
features_train.shape, features_test.shape, len(labels_train), len(labels_test)

((3900, 39213), (1672, 39213), 3900, 1672)

In [61]:
# Prediction using Support Vector Machine
svc = SVC(kernel='sigmoid', gamma=1.0)
svc.fit(features_train, labels_train)
prediction = svc.predict(features_test)
# accuracy_score(labels_test,prediction)
print(classification_report(labels_test, prediction, target_names = ['ham', 'spam']))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1440
        spam       0.99      0.85      0.91       232

    accuracy                           0.98      1672
   macro avg       0.98      0.93      0.95      1672
weighted avg       0.98      0.98      0.98      1672



In [62]:
# Prediction using Multinomial Naive Bayes Model
mnb = MultinomialNB(alpha=0.2)
mnb.fit(features_train, labels_train)
prediction = mnb.predict(features_test)
# accuracy_score(labels_test,prediction)
print(classification_report(labels_test, prediction, target_names = ['ham', 'spam']))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1440
        spam       0.96      0.93      0.95       232

    accuracy                           0.99      1672
   macro avg       0.97      0.96      0.97      1672
weighted avg       0.98      0.99      0.98      1672

